In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [2]:
# Hochladen beider .csv Dateien: kunden & besucher
kunden = pd.read_csv('final project/kunden.csv', sep = ';')
besucher = pd.read_csv('final project/besucher.csv', sep = ';')

# Hinzufügen einer neuen Spalte mit einem Standardwert
besucher['Preis'] = ''

# Anpassen der Spaltenreihenfolge, um 'Preis'-Spalte in besucher als dritte Spalte zu setzen
cols = list(besucher.columns)
cols.insert(2, cols.pop(cols.index('Preis')))
besucher = besucher[cols]

# Zusammenfügen der DataFrames untereinander (vertikal)
combined_draft = pd.concat([kunden, besucher], ignore_index=True, axis = 0)

# Sortieren des kombinierten DataFrames nach der Spalte 'KundeNr' aufsteigend
combined_sorted = combined_draft.sort_values(by='KundeNr')

# Gruppieren der Werte nach Überschrift
combined_gruppen = combined_draft.groupby('Geschlecht').size().reset_index(name='Anzahl')

# Zählen der Fehlerwerte in allen Spalten
na_count = combined_draft.isna().sum()

# Ausgabe des sortierten DataFrames
print(combined_draft)

      Alter          Einkommen     Preis  Geschlecht  Zeit KundeNr
0        64            66894.0  88160.31         1.0  43.0   K0310
1        54            77644.0  103145.7         1.0  40.0   K1042
2        55            44341.0  80565.16         0.0  37.0   K0382
3        49            67271.0  83949.89         0.0  42.0   K0498
4        46            49832.0  93781.58         0.0  41.0   K0552
...     ...                ...       ...         ...   ...     ...
1990     32  34391,82496545645                   1.0  48,0   K1211
1991     41  68516,49543941053                   0.0  34,0   K1375
1992     65  89685,33344089528                   0.0  23,0   K1965
1993     68  95714,30526812703                   0.0  32,0   K1540
1994     49   73319,1870845747                   0.0  27,0   K1207

[1995 rows x 6 columns]


In [3]:
land = pd.read_csv('final project/geo.txt', sep = '\t')

# Ersetzen der Werte in der Spalte "Niederlassung"
land['Niederlassung'] = land['Niederlassung'].replace({
    'NRW': 'Nordrhein-Westfalen',
    'Düsseldorf': 'Nordrhein-Westfalen',
    'Berlin-Charlottenburg': 'Berlin',
    'Berlin-Mitte': 'Berlin',
    'BERLIN': 'Berlin'
})

# Gruppieren nach der Spalte "Niederlassung" und Zählen der Einträge in jeder Gruppe/ Überprüfen auf Richtigkeit der Bundesländer
land_gruppen = land.groupby('Niederlassung').size().reset_index(name='Anzahl')

land

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,Nordrhein-Westfalen
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern
...,...,...
1990,K1991,Hessen
1991,K1992,Baden-Württemberg
1992,K1993,Nordrhein-Westfalen
1993,K1994,Berlin


In [4]:
# Anpassen der Spaltenreihenfolge, um "Niederlassung" als siebente (in Python sechste, weil Beginn = 0) Spalte zu setzen
cols = list(combined_draft.columns)
cols.insert(6, cols.pop(cols.index('KundeNr')))
combined = combined_draft[cols]

# Zusammenführen von "combined_draft" und "land", Sortierung nach "KundeNr" und Einfügen der "Niederlassung" Werte in Spalte "Niederlassung"
combined = pd.merge(combined_draft, land, on='KundeNr', how='right')

# Ersetzen von Kommas durch Punkte in allen Spalten
combined['Zeit'] = combined['Zeit'].astype(str).str.replace(',', '.', regex=False)
combined['Einkommen'] = combined['Einkommen'].astype(str).str.replace(',', '.', regex=False)
combined['Preis'] = combined['Preis'].astype(str).str.replace(',', '.', regex=False)
combined['Geschlecht'] = combined['Geschlecht'].astype(str).str.replace(',', '.', regex=False)

spalten_int = ['Einkommen', 'Preis']
for col in spalten_int:
    combined[col] = pd.to_numeric(combined[col], errors='coerce')

# Runden der Werte in 'Einkommen' und 'Preis'
combined['Einkommen'] = combined['Einkommen'].round(2)
combined['Preis'] = combined['Preis'].round(2)

# Speichern des aktualisierten DataFrames in eine neue CSV-Datei
# combined_df.to_csv('final_project/kunden_mit_niederlassung.csv', index=False)

spalten = ['Alter', 'Einkommen', 'Preis', 'Zeit']

# Berechnen der Z-Scores für die oben angegebenen Spalten
numeric_cols = combined.select_dtypes(include=np.number).columns
z_scores = combined[numeric_cols].apply(zscore)

# Ersetzen der Werte mit dem Medianwert, die einen Z-Score <3 oder >3 haben
for col in numeric_cols:
    median = combined[col].median()
    combined[col] = np.where(np.abs(z_scores[col]) > 3, median, combined[col])

# Ausgabe des Ergebnisses
combined

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Niederlassung
0,68.0,89437.00,114481.90,1.0,62.0,K0001,Sachsen
1,58.0,65271.00,106258.99,1.0,44.0,K0002,Nordrhein-Westfalen
2,44.0,48195.00,85705.57,1.0,29.0,K0003,Hessen
3,37.0,48957.00,76412.36,0.0,36.0,K0004,Bayern
4,50.0,56621.00,92898.67,0.0,40.0,K0005,Bayern
...,...,...,...,...,...,...,...
1990,48.0,59967.31,NaN,1.0,21.0,K1991,Hessen
1991,67.0,81521.76,NaN,1.0,21.0,K1992,Baden-Württemberg
1992,40.0,41616.57,NaN,0.0,48.0,K1993,Nordrhein-Westfalen
1993,49.0,76602.79,NaN,1.0,19.0,K1994,Berlin


# F1: Wie viele Autos wurden verkauft?

In [5]:
# Importieren der zusammengefügten Datei "combined.csv" und Aufteilen der Werte in Zeilen
# combined = pd.read_csv('combined.csv', sep = ',')

# Zählen der Preis und dadurch die Anzahl der verkauften Autos
verkaufte_autos  = combined['Preis'].count()

# Ausgeben des Ergebnisses
print(f"Es wurden insgesamt {verkaufte_autos} Autos verkauft.")

Es wurden insgesamt 1104 Autos verkauft.


# F4: Wie viele Autos wurden pro Bundesland verkauft?

In [6]:
# combined = pd.read_csv('combined.csv')

# Gruppieren der Bundesländer und Zählen der Preise, daher auch Anzahl verkaufter Autos. Zurücksetzen des Indexes.
verkäufe_pro_bundesland = combined.groupby('Niederlassung')['Preis'].count().reset_index()

# Umbenennen der Spalten "Niederlassung" und "Preis"
verkäufe_pro_bundesland = verkäufe_pro_bundesland.rename(columns={'Niederlassung': 'Bundesland', 'Preis': 'Verkaufte Autos'}).sort_values('Verkaufte Autos').reset_index(drop = True)

# Überschrift für das Ergebnis
print("Anzahl verkaufter Autos pro Bundesland:\n")

# Ausgeben des Ergebnisses und Entfernen des Indexes
print(verkäufe_pro_bundesland.to_string(index = False))

Anzahl verkaufter Autos pro Bundesland:

         Bundesland  Verkaufte Autos
        Brandenburg               37
            Hamburg               43
          Thüringen               49
            Sachsen               89
             Hessen               96
      Niedersachsen               96
             Berlin              126
Nordrhein-Westfalen              175
             Bayern              188
  Baden-Württemberg              205


# F7: Haben mehr Frauen oder mehr Männer unsere Autos gekauft?

In [7]:
# combined = pd.read_csv('combined.csv')

# Filtern nach "Preis", sodass nur die verkauften Autos betrachtet werden
verkaufte_autos = combined.dropna(subset=['Preis'])

# Anzahl der verkauften Autos nach Geschlecht zählen
geschlechter = verkaufte_autos['Geschlecht'].value_counts()

# Berechnung der Ergebnisse
maenner = geschlechter.get(1, 0)
frauen = geschlechter.get(0, 0)

# Ausgabe der Ergbenisse
print(f"Insgesamt haben {maenner} Männer unsere Autos gekauft.")
print(f"Insgesamt haben {frauen} Frauen unsere Autos gekauft.\n")

# Berechnung und Ausgabe des Ergebnisses, ob mehr männliche oder mehr weibliche Kunden
if maenner > frauen:
    print("Wir haben mehr männliche Kunden.")
elif frauen > maenner:
    print("Wir haben mehr weibliche Kunden.")
else:
    print("Unsere Kundschaft ist zu gleichen Teilen männlich und weiblich.")

Insgesamt haben 398 Männer unsere Autos gekauft.
Insgesamt haben 701 Frauen unsere Autos gekauft.

Wir haben mehr weibliche Kunden.


# F13: Wie viele Kunden haben keinen Bankkredit aufgenommen, um das Auto zu kaufen?

In [8]:
# combined = pd.read_csv('combined.csv')

# Annahme: Kunden, deren Gehalt höher ist als der Kaufpreis nehmen keinen Kredit auf
kunden_mit_kredit = combined[combined['Einkommen'] < combined['Preis']]

# Kunden die einen Kredit aufgenommen haben zählen
kredite = len(kunden_mit_kredit)

# Ausgabe des Ergebnisses
print(f"Es haben insgesamt {kredite} Kunden einen Kredit für den Kauf aufgenommen.")

Es haben insgesamt 1099 Kunden einen Kredit für den Kauf aufgenommen.


# F16: Welche Variable beeinflusst den Preis des Autos am meisten?

In [9]:
# combined = pd.read_csv('combined.csv')

# Entfernen von Zeilen mit fehlenden Werten in den relevanten Spalten
combined = combined.dropna()

# Entfernen von negativen Werten in den Spalten "Einkommen" und "Preis"
combined = combined[(combined['Einkommen'] >= 0) & (combined['Preis'] >= 0)]

# Konvertieren von Kategorien in numerische Werte (falls notwendig)
if 'Geschlecht' in combined.columns:
    combined = pd.get_dummies(combined, columns=['Geschlecht'], drop_first=True)
else:
    print("Warnung: Die Spalte 'Geschlecht' ist nicht im DataFrame enthalten.")

# Auswahl der Merkmale und der Zielvariable
features = ['Alter', 'Einkommen'] + [col for col in combined.columns if 'Geschlecht' in col]
target = 'Preis'
X = combined[features]
y = combined[target]

# Standardisieren der Merkmale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Durchführen der Regressionsanalyse
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
#print(f'Modell R^2 Score: {score}')

# Regressionskoeffizienten anzeigen
coefficients = model.coef_
coeff_df = pd.DataFrame(coefficients, index=features, columns=['Koeffizient'])
print(coeff_df.round(2))

# Interpretieren der Regressionskoeffizienten
max_coefficient = coeff_df.max().round(2)[0]
print()
print(f"Die Variable 'Einkommen' hat mit einem Wert von {max_coefficient} den höchsten absoluten Koeffizientenwert und beeinflusst damit den Preis des Autos am meisten.")

                Koeffizient
Alter               6612.70
Einkommen          16487.79
Geschlecht_1.0      1146.39
Geschlecht_nan        -6.25

Die Variable 'Einkommen' hat mit einem Wert von 16487.79 den höchsten absoluten Koeffizientenwert und beeinflusst damit den Preis des Autos am meisten.


# F19:Auf welche Probleme sind Sie bei der Zusammenführung des Datensatzes gestoßen?

In [10]:
print("Folgende Probleme sind aufgetreten:\n\n"
     
    "1. Einfügen einer fehlenden Spalte: besucher['Preis'] = ''\n\n"
    "2. Einfügen der Spalte und Setzen an richtige Stelle: cols.insert()\n\n"
    "3. Ersetzen der falsch bezeichneten Bundesländer: land['Niederlassung'].replace()\n\n"
    "4. Zusammenführen der .csv und -txt Dateien: combined = pd.merge(combined_draft, land, on='KundeNr', how='right')\n\n"
    "5. Ersetzen von ',' in '.': combined['Zeit'].astype(str).str.replace(',', '.', regex=False)\n\n"
    "6. Umwandeln der Werte in numerische Werte: combined[col] = pd.to_numeric()\n\n"
    "7. Ersetzen der Ausreißer durch den Medianwert: combined[col] = np.where(np.abs(z_scores[col]) > 3, median, combined[col])")

Folgende Probleme sind aufgetreten:

1. Einfügen einer fehlenden Spalte: besucher['Preis'] = ''

2. Einfügen der Spalte und Setzen an richtige Stelle: cols.insert()

3. Ersetzen der falsch bezeichneten Bundesländer: land['Niederlassung'].replace()

4. Zusammenführen der .csv und -txt Dateien: combined = pd.merge(combined_draft, land, on='KundeNr', how='right')

5. Ersetzen von ',' in '.': combined['Zeit'].astype(str).str.replace(',', '.', regex=False)

6. Umwandeln der Werte in numerische Werte: combined[col] = pd.to_numeric()

7. Ersetzen der Ausreißer durch den Medianwert: combined[col] = np.where(np.abs(z_scores[col]) > 3, median, combined[col])
